In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/german-credit-data-with-risk/german_credit_data.csv', index_col='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
y = df['Risk'].replace({'good': 1, 'bad': 0})
X = df.drop('Risk', axis=1)

In [ ]:
df.info()

In [ ]:
obj_cols = df.select_dtypes('object').columns
df[obj_cols].nunique()

In [ ]:
print('saving accounts',  df['Saving accounts'].unique())
print('Checking account', df['Checking account'].unique())

In [ ]:
X['Saving accounts'] = pd.Categorical(X['Saving accounts'], categories=['little', 'moderate', 'rich', 'quite rich'], ordered=True)
X['Checking account'] = pd.Categorical(X['Checking account'], categories=['little', 'moderate', 'rich'], ordered=True)

In [ ]:
obj_cols = X.select_dtypes('object').columns
df[obj_cols].nunique()

In [ ]:
X[obj_cols] = X[obj_cols].astype('category')

In [ ]:
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from statsmodels.nonparametric.smoothers_lowess import lowess

In [ ]:
Xt, Xv, yt, yv = train_test_split(X, y, random_state=0)
dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

In [ ]:
import lightgbm as lgb

In [ ]:
def loguniform(low=0, high=1, size=None, base=10):
    return np.power(base, np.random.uniform(low, high, size))

In [ ]:
best_etas = {"eta": [], "score": []}

In [ ]:
metric = "binary_logloss"
for _ in range(100):
    eta = loguniform(-4, 0)
    best_etas["eta"].append(eta)
    model = lgb.train(
        {"objective": "binary", "eta": eta},
        dt,
        num_boost_round=10000,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    best_etas["score"].append(model.best_score["valid"][metric])

best_eta_df = pd.DataFrame.from_dict(best_etas)
lowess_data = lowess(best_eta_df["score"], best_eta_df["eta"],)

plt.xscale("log")
rounded_data = lowess_data.copy()
rounded_data[:, 1] = rounded_data[:, 1].round(4)
rounded_data = rounded_data[::-1]  # reverse to find first minimum
good_enough_eta = rounded_data[rounded_data[:, 1].argmin(), 0]

# plot relationship between learning rate and performance, with its corresponding optimal eta
print(f"Good learning rate: {good_enough_eta}")
plt.axvline(good_enough_eta, color="orange")
plt.title("Smoothed relationship between learning rate and cost.")
plt.xlabel("learning rate")
plt.ylabel(metric)
sns.lineplot(lowess_data[:, 0], lowess_data[:, 1])

In [ ]:
model = lgb.train(
    {"objective": "binary", "eta": good_enough_eta},
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=5,
)

In [ ]:
threshold = 0.75
corr = Xt.corr(method="kendall")
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
upper = upper.stack()
high_upper = upper[(abs(upper) > threshold)]
abs_high_upper = abs(high_upper).sort_values(ascending=False)
pairs = abs_high_upper.index.to_list()
print(f"Correlated features: {pairs if len(pairs) > 0 else None}")

In [ ]:
sorted_features = [
    feature
    for _, feature in sorted(
        zip(model.feature_importance(), dt.feature_name), reverse=False
    )
]

best_score = model.best_score["valid"][metric]
print(f"starting score: {best_score:.4f}")
drop_unimportant_features = []
for feature in sorted_features:
    drop_unimportant_features.append(feature)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X.drop(drop_unimportant_features, axis=1), y, random_state=0
    )
    dt = lgb.Dataset(X_train, y_train)
    dv = lgb.Dataset(X_valid, y_valid)
    drop_model = lgb.train(
        {"objective": "binary", "metric": metric, "eta": good_enough_eta},
        dt,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        num_boost_round=10000,
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    score = drop_model.best_score["valid"][metric]
    if score > best_score:
        del drop_unimportant_features[-1]  # remove from drop list
        print(f"Dropping {feature} worsened score to {score:.4f}.")
        break
    else:
        best_score = score
print(f"ending score: {best_score:.4f}")
print(
    f"dropped features: {drop_unimportant_features if len(drop_unimportant_features) > 0 else None}"
)

In [ ]:
import optuna.integration.lightgbm as lgb

params = {
    "objective": "binary",
    "metric": metric,
    "verbosity": -1,
    "boosting_type": "gbdt",
    "eta": good_enough_eta,
}

dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)


model = lgb.train(
    params,
    dt,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    num_boost_round=10000,
    verbose_eval=False,
    early_stopping_rounds=50,
)

score = model.best_score["valid"][metric]

best_params = model.params
print("Best params:", best_params)
print(f"  {metric} = {score}")
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

In [ ]:
import lightgbm as lgb

In [ ]:
best_params['eta'] = good_enough_eta

In [ ]:
model = lgb.train(
best_params,
dt,
num_boost_round=10000,
valid_sets=[dt, dv],
valid_names=["training", "valid"],
early_stopping_rounds=50,
verbose_eval=5,
)

In [ ]:
sorted_features = [
    feature
    for _, feature in sorted(
        zip(model.feature_importance(), dt.feature_name), reverse=False
    )
]

best_score = model.best_score["valid"][metric]
print(f"starting score: {best_score:.4f}")
drop_unimportant_features = []
for feature in sorted_features:
    drop_unimportant_features.append(feature)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X.drop(drop_unimportant_features, axis=1), y, random_state=0
    )
    dt = lgb.Dataset(X_train, y_train)
    dv = lgb.Dataset(X_valid, y_valid)
    drop_model = lgb.train(
        best_params,
        dt,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        num_boost_round=10000,
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    score = drop_model.best_score["valid"][metric]
    if score > best_score:
        del drop_unimportant_features[-1]  # remove from drop list
        print(f"Dropping {feature} worsened score to {score:.4f}.")
        break
    else:
        best_score = score
print(f"ending score: {best_score:.4f}")
print(
    f"dropped features: {drop_unimportant_features if len(drop_unimportant_features) > 0 else None}"
)

In [ ]:
lgb.plot_importance(model, grid=False)

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
ypred = model.predict(Xv, num_iteration=model.best_iteration)
print(f"AUC: {roc_auc_score(yv, ypred):.4f}")
print(classification_report(yv, ypred > 0.5))